# In-Vantage Scoring a través de PMML

Como parte del modelo de RRHH Accidentalidad, se genera una representación PMML del modelo de regresión logística gracias al código

```python
from nyoka import skl_to_pmml
...
skl_to_pmml(pipeline=model, col_names=feature_names, target_name=target_name, pmml_f_name="artifacts/output/model.pmml")
```

Se puede desplegar esto en modo In-Vantage a través de la interfaz gráfica de AnalyticOps, de la misma forma que los modos RESTFul y Batch.
Este notebook muestra como se puede usar este modo para realizar scoring de modelos por medio de consultas SQL.

In [1]:
import os
import pandas as pd
import getpass

from teradataml import create_context, remove_context
from teradataml.context.context import get_connection

In [2]:
password = getpass.getpass("Contraseña")

engine = create_context(host="40.117.178.55", username="aoa_service", password=password, database="aoa_data")
conn = get_connection()
cursor = engine.raw_connection().cursor()

Contraseña ···········


## Ver los modelos publicados

Es posible ver los modelos publicados en **Vantage** consultando la tabla en donde están publicados. Nota: esta información también está disponible a través del API de AnalyticOps.

In [3]:
pd.options.display.max_colwidth = 250
pd.read_sql("select * from aoa_data.aoa_ivsm_models", conn)

,model_version,model_id,model_type,project_id,deployed_at,model
0,dbc5f29e-ac76-441e-8025-9344307f635f,f937b5d8-02c6-5150-80c7-1e4ff07fea31,PMML,23e1df4b-b630-47a1-ab80-7ad5385fcd8d,2022-01-10 19:08:24.130,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."
1,df300e1a-781b-4b3e-9b8e-f32ef54c7a89,73d3f5a1-a5a9-55fd-9fe8-298a8aa7929d,PMML,a85ede91-9b28-4a7d-bb32-eee2aaffa82f,2022-01-10 19:09:58.490,"b'<?xml version=""1.0"" encoding=""UTF-8""?>\n<PMML xmlns=""http://www.dmg.org/PMML-4_4"" version=""4.4"">\n <Header copyright=""Copyright (c) 2018 Software AG"" description=""Default Description"">\n <Application name=""Nyoka"" version=""4.3.0""/>\n ..."


## Scoring bajo demanda

Para realizar scoring bajo demanda, solo hay que ejecutar una consulta SQL similar a la que se muestra a continuación, donde se debe definir la tabla a predecir, el identificador del modelo y la versión del modelo.

In [4]:
query="""
SELECT * FROM ivsm.IVSM_SCORE(
    on ACC_TEST
    on (SELECT model_id, model FROM aoa_data.aoa_ivsm_models WHERE model_version = 'df300e1a-781b-4b3e-9b8e-f32ef54c7a89') AS ModelTable DIMENSION
    using
        ModelID('73d3f5a1-a5a9-55fd-9fe8-298a8aa7929d')
        ColumnsToPreserve('id')
        ModelType('PMML')
) sc;
"""
pd.read_sql(query, conn).head()

,id,score_result
0,762,"{""probability_0"":0.7702602150213241,""probability_1"":0.22973978497867584,""predicted_ptarget"":0,""probability"":0.7702602150213241}"
1,763,"{""probability_0"":0.27459126505508746,""probability_1"":0.7254087349449125,""predicted_ptarget"":1,""probability"":0.7254087349449125}"
2,764,"{""probability_0"":0.31745134809519304,""probability_1"":0.682548651904807,""predicted_ptarget"":1,""probability"":0.682548651904807}"
3,765,"{""probability_0"":0.9271389074598685,""probability_1"":0.07286109254013155,""predicted_ptarget"":0,""probability"":0.9271389074598685}"
4,423,"{""probability_0"":0.5675869655216632,""probability_1"":0.4324130344783368,""predicted_ptarget"":0,""probability"":0.5675869655216632}"


Copyright 2021 Teradata. All rights reserved.